In [31]:
database_folder = "spider_data/test_database"
query_file_path = "spider_data/test.json"

schema_output_path = "prepare_data/test_schemas.csv"
final_result_path = "prepare_data/test_input.csv"

In [32]:
import os
import sys

sys.path.append(os.getcwd() + "/M-Schema")

from schema_engine import SchemaEngine

### Load full database schemas and tables

In [33]:
import os
import sqlite3
import pandas as pd
from sqlalchemy import create_engine


def gather_schemas(root_dir: str) -> pd.DataFrame:
    """
    Walk root_dir for .sqlite files. For each, connect and extract
    each table's CREATE statement. Returns a DataFrame with columns:
      - db_id : filename without .sqlite
      - table : table name
      - schema: CREATE TABLE ... statement
    """
    records = []

    for dirpath, _, filenames in os.walk(root_dir):
        for fn in filenames:
            if fn.lower().endswith(".sqlite"):
                path = os.path.join(dirpath, fn)
                db_id = os.path.splitext(fn)[0]
                # connect to sqlite
                conn = sqlite3.connect(path)
                try:
                    cursor = conn.cursor()
                    # get list of tables
                    cursor.execute(
                        "SELECT name FROM sqlite_master WHERE type='table' AND name NOT LIKE 'sqlite_%';"
                    )
                    tables = [row[0] for row in cursor.fetchall()]

                    for table in tables:
                        # get CREATE statement
                        cursor.execute(
                            "SELECT sql FROM sqlite_master WHERE type='table' AND name=?;",
                            (table,),
                        )
                        row = cursor.fetchone()
                        schema = row[0] if row and row[0] else None
                        cursor.execute(f"SELECT * FROM {table} LIMIT 3;")
                        ex_rows = cursor.fetchall()
                        ex_rows_str = f"Table: {table}\n" + "\n".join(
                            [str(row) for row in ex_rows]
                        )
                        schema_engine = SchemaEngine(
                            engine=create_engine(f"sqlite:///{path}"), db_name=db_id
                        )
                        mschema = schema_engine.mschema
                        mschema_str = mschema.to_mschema()
                        records.append(
                            {
                                "db_id": db_id,
                                "table": table,
                                "schema": schema,
                                "example_rows": ex_rows_str,
                                "mschema": mschema_str,
                            }
                        )
                finally:
                    conn.close()

    return pd.DataFrame(records)

In [ ]:
schema_df = gather_schemas(database_folder)
schema_df

In [35]:
schema_df.to_csv(schema_output_path)

In [36]:
schemas_map = {}
example_rows_map = {}
mschemas_map = {}
for _, row in schema_df.iterrows():
    schemas_map.setdefault(row.db_id, {})[row.table.lower()] = row.schema
    example_rows_map.setdefault(row.db_id, {})[row.table.lower()] = row.example_rows
    mschemas_map.setdefault(row.db_id, {})[row.table.lower()] = row.mschema

### Parse queries

In [37]:
import re
from typing import List, Set

import sqlglot
from sqlglot import exp


def extract_tables(sql: str) -> List[str]:
    """
    Parse the given SQL and return a sorted list of all
    real table names referenced (joins, subqueries, CTE bodies, etc.),
    but *exclude* any CTE aliases.
    """
    # 1) Pull out the CTE block (if any) and collect its aliases
    cte_names: Set[str] = set()
    match = re.search(
        r"WITH\s+(.*?)\)\s*SELECT",
        sql,
        flags=re.IGNORECASE | re.DOTALL,
    )
    if match:
        cte_block = match.group(1)
        # find all "<name> AS (" inside that block
        found = re.findall(
            r"([A-Za-z_][A-Za-z0-9_]*)\s+AS\b", cte_block, flags=re.IGNORECASE
        )
        cte_names = set(found)

    # 2) Parse into an AST
    try:
        tree = sqlglot.parse_one(sql)
    except sqlglot.errors.ParseError as e:
        raise ValueError(f"Failed to parse SQL: {e}")

    # 3) Walk every Table node and collect its .this (the table identifier),
    #    unless it’s one of the CTE names we just saw.
    tables: Set[str] = set()
    for tbl in tree.find_all(exp.Table):
        name = tbl.name  # e.g. 'sales', 'customers', 'archived_sales'
        if name not in cte_names:
            tables.add(name)

    return sorted(tables)


def lookup_schemas(row):
    db = row.db_id
    tables = row.tables  # list of table names
    db_map = schemas_map.get(db, {})
    # collect schema for each table, skip if not found
    return [db_map.get(tbl.lower(), f"<no schema for {tbl}>") for tbl in tables]


def lookup_example_rows(row):
    db = row.db_id
    tables = row.tables
    db_map = example_rows_map.get(db, {})
    return [db_map.get(tbl.lower(), f"<no example rows for {tbl}>") for tbl in tables]


def lookup_mschemas(row):
    db = row.db_id
    tables = row.tables
    db_map = mschemas_map.get(db, {})
    return [db_map.get(tbl.lower(), f"<no mschema for {tbl}>") for tbl in tables]

In [38]:
import pandas as pd

df = pd.read_json(query_file_path)
df = (
    df.assign(
        question_number=df.index,
        tables=lambda df_: df_["query"].apply(extract_tables),
    )
    .assign(
        schemas=lambda df_: df_.apply(lookup_schemas, axis=1),
        example_rows=lambda df_: df_.apply(lookup_example_rows, axis=1),
        mschemas=lambda df_: df_.apply(lookup_mschemas, axis=1),
    )
    .loc[
        :,
        [
            "question_number",
            "question",
            "db_id",
            "tables",
            "schemas",
            "example_rows",
            "mschemas",
        ],
    ]
)
df

,question_number,question,db_id,tables,schemas,example_rows,mschemas
0,0,How many clubs are there?,soccer_3,[club],"[CREATE TABLE ""club"" (\n""Club_ID"" int,\n""Name""...","[Table: club\n(1, 'Arsenal', 'Arsène Wenger', ...",[【DB_ID】 soccer_3\n【Schema】\n# Table: main.clu...
1,1,Count the number of clubs.,soccer_3,[club],"[CREATE TABLE ""club"" (\n""Club_ID"" int,\n""Name""...","[Table: club\n(1, 'Arsenal', 'Arsène Wenger', ...",[【DB_ID】 soccer_3\n【Schema】\n# Table: main.clu...
2,2,List the name of clubs in ascending alphabetic...,soccer_3,[club],"[CREATE TABLE ""club"" (\n""Club_ID"" int,\n""Name""...","[Table: club\n(1, 'Arsenal', 'Arsène Wenger', ...",[【DB_ID】 soccer_3\n【Schema】\n# Table: main.clu...
3,3,"What are the names of clubs, ordered alphabeti...",soccer_3,[club],"[CREATE TABLE ""club"" (\n""Club_ID"" int,\n""Name""...","[Table: club\n(1, 'Arsenal', 'Arsène Wenger', ...",[【DB_ID】 soccer_3\n【Schema】\n# Table: main.clu...
4,4,What are the managers and captains of clubs?,soccer_3,[club],"[CREATE TABLE ""club"" (\n""Club_ID"" int,\n""Name""...","[Table: club\n(1, 'Arsenal', 'Arsène Wenger', ...",[【DB_ID】 soccer_3\n【Schema】\n# Table: main.clu...
...,...,...,...,...,...,...,...
2142,2142,Return the ids and details of staff who have a...,advertising_agencies,[staff],[CREATE TABLE `Staff` (\n`staff_id` INTEGER PR...,"[Table: Staff\n(1, 6, 'Rubie')\n(2, 7, 'Sherid...",[【DB_ID】 advertising_agencies\n【Schema】\n# Tab...
2143,2143,"What are the id, sic code and agency id of the...",advertising_agencies,"[clients, invoices, meetings]",[CREATE TABLE `Clients` (\n`client_id` INTEGER...,"[Table: Clients\n(1, 8, 'Mutual', 'Alta')\n(2,...",[【DB_ID】 advertising_agencies\n【Schema】\n# Tab...
2144,2144,"Return the ids, sic codes, and agency ids of c...",advertising_agencies,"[clients, invoices, meetings]",[CREATE TABLE `Clients` (\n`client_id` INTEGER...,"[Table: Clients\n(1, 8, 'Mutual', 'Alta')\n(2,...",[【DB_ID】 advertising_agencies\n【Schema】\n# Tab...
2145,2145,"List the start time, end time of each meeting,...",advertising_agencies,"[clients, meetings, staff, staff_in_meetings]",[CREATE TABLE `Clients` (\n`client_id` INTEGER...,"[Table: Clients\n(1, 8, 'Mutual', 'Alta')\n(2,...",[【DB_ID】 advertising_agencies\n【Schema】\n# Tab...


In [39]:
df.to_csv(final_result_path, index=False)